<a href="https://colab.research.google.com/github/FernandoSimon22/vision_artificial/blob/main/Faster_R_CNN_implementacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from PIL import Image

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lX9sE2RPQOCv2R7cn0io")
project = rf.workspace("jota22").project("am_boundingbox")
version = project.version(14)
dataset = version.download("coco")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define transformations
class CocoTransform:
    def __call__(self, image, target):
        image = F.to_tensor(image)  # Convert PIL image to tensor
        return image, target

In [ ]:
# Dataset class
def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )

# Load datasets
train_dataset = get_coco_dataset(
    img_dir="/content/AM_BoundingBox-14/train",
    ann_file="/content/AM_BoundingBox-14/train/_annotations.coco.json"
)


val_dataset = get_coco_dataset(
    img_dir="/content/AM_BoundingBox-14/valid",
    ann_file="/content/AM_BoundingBox-14/valid/_annotations.coco.json"
)



# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
# Load Faster R-CNN with ResNet-50 backbone
def get_model(num_classes):
    # Load pre-trained Faster R-CNN
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
# Initialize the model
num_classes = 2 # background + def
model = get_model(num_classes)

In [ ]:
# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer and learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    for images, targets in data_loader:
        # Move images to the device
        images = [img.to(device) for img in images]

        # Validate and process targets
        processed_targets = []
        valid_images = []
        for i, target in enumerate(targets):
            boxes = []
            labels = []
            for obj in target:
                # Extract bbox
                bbox = obj["bbox"]  # Format: [x, y, width, height]
                x, y, w, h = bbox

                # Ensure the width and height are positive
                if w > 0 and h > 0:
                    boxes.append([x, y, x + w, y + h])  # Convert to [x_min, y_min, x_max, y_max]
                    labels.append(obj["category_id"])

            # Only process if there are valid boxes
            if boxes:
                processed_target = {
                    "boxes": torch.tensor(boxes, dtype=torch.float32).to(device),
                    "labels": torch.tensor(labels, dtype=torch.int64).to(device),
                }
                processed_targets.append(processed_target)
                valid_images.append(images[i])  # Add only valid images

        # Skip iteration if no valid targets
        if not processed_targets:
            continue

        # Ensure images and targets are aligned
        images = valid_images

        # Forward pass
        loss_dict = model(images, processed_targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch [{epoch}] Loss: {losses.item():.4f}")

In [ ]:
# Training loop
num_epochs = 150
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()

    # Save the model's state dictionary every 10 epochs
    if (epoch + 1) % 15 == 0:
        model_path = f"fasterrcnn_resnet50_epoch_{epoch + 1}.pth"
        torch.save(model.state_dict(), model_path)
        print(f"✅ Model saved: {model_path}")


In [ ]:
# --- MEJORADO: Evaluar todas las imágenes del conjunto de validación y visualizar ---

import os
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms

# --- CONFIGURACIÓN ---
num_classes = 2  # 1 clase + fondo
model_path = "/content/fasterrcnn_resnet50_epoch_150.pth"
img_dir = "/content/AM_BoundingBox-14/test"  # <-- Cambia esto si es necesario
ann_file = "/content/AM_BoundingBox-14/test/_annotations.coco.json"

# --- CARGAR MODELO ---
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model(num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# --- COCO DATASET & LOADER ---
def get_transform():
    return transforms.Compose([
        transforms.ToTensor()
    ])

def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(root=img_dir, annFile=ann_file, transform=get_transform())

dataset = get_coco_dataset(img_dir, ann_file)
val_loader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# --- FUNCIONES DE DIBUJO ---
COCO_CLASSES = {0: "background", 1: "def"}

def get_class_name(class_id):
    return COCO_CLASSES.get(class_id, "unknown")

def draw_boxes(image, prediction, threshold=0.5):
    draw = ImageDraw.Draw(image)
    boxes = prediction["boxes"].cpu().numpy()
    labels = prediction["labels"].cpu().numpy()
    scores = prediction["scores"].cpu().numpy()

    for box, label, score in zip(boxes, labels, scores):
        if score >= threshold:
            x_min, y_min, x_max, y_max = box
            draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
            draw.text((x_min, y_min), f"{get_class_name(label)} ({score:.2f})", fill="red")
    return image

# --- EVALUAR Y GUARDAR RESULTADOS DE TODAS LAS IMÁGENES ---
out_dir = "detections"
os.makedirs(out_dir, exist_ok=True)

with torch.no_grad():
    for idx, (images, targets) in enumerate(tqdm(val_loader, desc="Evaluando")):
        image_tensor = images[0].to(device)
        output = model([image_tensor])[0]

        # Cargar imagen original para visualizar
        image_path = dataset.ids[idx]
        image_file = dataset.coco.loadImgs(image_path)[0]['file_name']
        image = Image.open(os.path.join(img_dir, image_file)).convert("RGB")

        # Dibujar predicciones
        result_img = draw_boxes(image.copy(), output)

        # Guardar
        result_img.save(os.path.join(out_dir, f"prediction_{idx+1}.jpg"))

FileNotFoundError: [Errno 2] No such file or directory: '/content/fasterrcnn_resnet50_epoch_150.pth'

In [ ]:
# --- FUNCIÓN PARA EVALUAR UN MODELO SOBRE TODO EL DATASET Y DEVOLVER MÉTRICAS ---

import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import os
from PIL import Image, ImageDraw
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt


def evaluate_model(model, val_loader, annotations_path, output_json="predictions.json", device="cuda"):
    model.eval()
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    model.to(device)

    results = []
    total_box_loss = 0.0
    total_cls_loss = 0.0
    n_batches = 0

    # Recorrer dataset de validación
    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc="Evaluando modelo"):
            images = [img.to(device) for img in images]
            new_targets = []
            for t in targets:
                if isinstance(t, list) and isinstance(t[0], dict):
                    new_targets.append({k: v.to(device) if torch.is_tensor(v) else v for k, v in t[0].items()})
                elif isinstance(t, dict):
                  # Create a 'boxes' key with the 'bbox' values
                    t['boxes'] = torch.tensor(t['bbox'], dtype=torch.float32).to(device)  # Assuming bbox is already a list
                    new_targets.append({k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()})
                else:
                    raise TypeError("Formato de target no reconocido: se esperaba lista de dicts o dict")

            targets = new_targets

            # Calcular pérdidas (modo train necesario)
            model.train()
            loss_dict = model(images, targets)
            total_box_loss += loss_dict['loss_box_reg'].item()
            total_cls_loss += loss_dict['loss_classifier'].item()
            model.eval()

            outputs = model(images)

            for i in range(len(outputs)):
                image_id = targets[i]["image_id"].item()
                boxes = outputs[i]["boxes"].cpu().numpy()
                scores = outputs[i]["scores"].cpu().numpy()
                labels = outputs[i]["labels"].cpu().numpy()

                for box, score, label in zip(boxes, scores, labels):
                    x1, y1, x2, y2 = box
                    results.append({
                        "image_id": image_id,
                        "category_id": int(label),
                        "bbox": [x1, y1, x2 - x1, y2 - y1],
                        "score": float(score)
                    })

            n_batches += 1

    # Guardar predicciones
    with open(output_json, "w") as f:
        json.dump(results, f)

    # Evaluar con COCO
    coco_gt = COCO(annotations_path)
    coco_dt = coco_gt.loadRes(output_json)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return {
        "map": coco_eval.stats[0],
        "ap50": coco_eval.stats[1],
        "recall": coco_eval.stats[8],
        "box_loss": total_box_loss / n_batches,
        "cls_loss": total_cls_loss / n_batches
    }


In [ ]:
# --- COMPARAR VARIOS MODELOS .PTH ---
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def compare_models(model_paths, model, val_loader, annotations_path, device="cuda"):
    resultados = []
    for path in model_paths:
        model.load_state_dict(torch.load(path, map_location=device))
        print(f"\nEvaluando modelo: {path}")
        metrics = evaluate_model(model, val_loader, annotations_path, output_json="predictions_temp.json", device=device)
        resultados.append({
            "modelo": os.path.basename(path),
            "mAP@[.5:.95]": metrics["map"],
            "AP50": metrics["ap50"],
            "Recall": metrics["recall"],
            "Box Loss": metrics["box_loss"],
            "Cls Loss": metrics["cls_loss"]
        })

    df = pd.DataFrame(resultados)
    df_ordenado = df.sort_values(by="mAP@[.5:.95]", ascending=False)
    print("\n📊 Comparación de modelos:")
    print(df_ordenado.to_string(index=False))

    # --- GRAFICAR RESULTADOS ---
    plt.figure(figsize=(12, 6))
    plt.plot(df_ordenado['modelo'], df_ordenado['mAP@[.5:.95]'], label='mAP@[.5:.95]', marker='o')
    plt.plot(df_ordenado['modelo'], df_ordenado['AP50'], label='AP50', marker='x')
    plt.plot(df_ordenado['modelo'], df_ordenado['Recall'], label='Recall', marker='s')
    plt.xlabel("Modelo (.pth)")
    plt.ylabel("Métrica")
    plt.title("Comparación de métricas entre modelos")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # --- GUARDAR RESULTADOS EN CSV ---
    df_ordenado.to_csv("resultados_comparacion_modelos.csv", index=False)
    print("\n✅ Resultados exportados a 'resultados_comparacion_modelos.csv'")

    return df_ordenado


# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    # Lista de modelos a comparar
    model_paths = [
        "fasterrcnn_resnet50_epoch_105.pth",
        "fasterrcnn_resnet50_epoch_120.pth",
        "fasterrcnn_resnet50_epoch_135.pth"
    ]

    # Crear modelo base
    def get_model(num_classes):
        model = fasterrcnn_resnet50_fpn(pretrained=True)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model

    model = get_model(num_classes= 2)  # Ajusta según tu dataset
    device = "cuda" if torch.cuda.is_available() else "cpu"

     # Preparar val_loader
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    val_dataset = CocoDetection(
        root="/content/AM_BoundingBox-14/valid",
        annFile="/content/AM_BoundingBox-14/valid/_annotations.coco.json",
        transform=transform
    )

    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

    # Ruta a las anotaciones COCO del conjunto de validación
    annotations_path = "/content/AM_BoundingBox-14/valid/_annotations.coco.json"

    # Ejecutar comparación
    df_resultados = compare_models(model_paths, model, val_loader, annotations_path, device=device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


NameError: name 'torch' is not defined

In [ ]:
 # --- GUARDAR RESULTADOS EN CSV ---
    df_ordenado.to_csv("resultados_comparacion_modelos.csv", index=False)
    print("\n✅ Resultados exportados a 'resultados_comparacion_modelos.csv'")

    return df_ordenado